In [ ]:
import numpy as np

def gd_matrix_factorization(R, mask, k=10, epochs=100, alpha=0.001, tol=1e-6):
    """
    Implementazione del Gradient Descent per la Matrix Factorization
    secondo il pseudocodice fornito.

    Parametri
    ---------
    R : np.ndarray
        Matrice dei rating (dimensioni: num_users x num_items). 
        Le celle mancanti devono essere 0 (o un valore neutro), 
        mentre mask le esclude dal calcolo dell'errore.
    mask : np.ndarray
        Matrice binaria (stesse dimensioni di R) con 1 sulle entry osservate, 
        0 sulle mancanti.
    k : int
        Numero di fattori latenti.
    epochs : int
        Numero di iterazioni del gradiente.
    alpha : float
        Learning rate.
    tol : float
        Tolleranza per il criterio di convergenza sul costo (opzionale).

    Ritorna
    -------
    U : np.ndarray
        Matrice dei fattori degli utenti (dimensioni: num_users x k).
    V : np.ndarray
        Matrice dei fattori degli item (dimensioni: num_items x k).
    """

    num_users, num_items = R.shape

    # 1) Inizializzazione casuale delle matrici U e V
    U = np.random.rand(num_users, k)
    V = np.random.rand(num_items, k)

    # Funzione di costo iniziale (opzionale, per monitorare convergenza)
    prev_cost = None

    for epoch in range(epochs):
        # 2) Calcola e_ij = r_ij - u_i dot v_j per tutte le entry osservate
        #    Creiamo una matrice degli errori E della stessa dimensione di R
        E = np.zeros_like(R)

        # Popoliamo E solo per (i, j) dove mask[i, j] = 1
        for i in range(num_users):
            for j in range(num_items):
                if mask[i, j] == 1:  # (i, j) in S
                    E[i, j] = R[i, j] - np.dot(U[i, :], V[j, :])

        # 3) Aggiorna U (per ogni user i e componente q)
        for i in range(num_users):
            for q in range(k):
                # Somma su tutti gli item j osservati per l'utente i
                grad_u_iq = 0.0
                for j in range(num_items):
                    if mask[i, j] == 1:  # (i, j) in S
                        grad_u_iq += E[i, j] * V[j, q]
                U[i, q] += alpha * grad_u_iq

        # 4) Aggiorna V (per ogni item j e componente q)
        for j in range(num_items):
            for q in range(k):
                # Somma su tutti gli utenti i che hanno rating osservato per item j
                grad_v_jq = 0.0
                for i in range(num_users):
                    if mask[i, j] == 1:  # (i, j) in S
                        grad_v_jq += E[i, j] * U[i, q]
                V[j, q] += alpha * grad_v_jq

        # 5) Calcolo del costo (opzionale) e check di convergenza
        #    Il costo J = 1/2 * sum((r_ij - u_i dot v_j)^2) su (i, j) in S
        cost = 0.0
        for i in range(num_users):
            for j in range(num_items):
                if mask[i, j] == 1:
                    cost += (R[i, j] - np.dot(U[i, :], V[j, :]))**2
        cost *= 0.5

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, costo: {cost:.4f}")

        # Criterio di convergenza: se la riduzione del costo è < tol
        if prev_cost is not None and abs(prev_cost - cost) < tol:
            print(f"Convergenza raggiunta all'epoca {epoch} (costo ~ {cost:.4f})")
            break
        prev_cost = cost

    return U, V


# Esempio d'uso:
if __name__ == "__main__":
    # Supponendo di avere R e mask già pronti
    # (num_users x num_items)
    # R contiene 0 sulle entry mancanti, e mask=1 su quelle osservate
    R = np.array([
        [5, 3, 0, 1],
        [4, 0, 0, 1],
        [1, 1, 0, 5],
        [0, 0, 5, 4]
    ], dtype=float)
    mask = (R != 0).astype(float)

    # Parametri
    k = 2
    epochs = 10000
    alpha = 0.0001

    U, V = gd_matrix_factorization(R, mask, k, epochs, alpha)

    # Ricostruzione della matrice
    R_hat = U.dot(V.T)
    print("\nMatrice ricostruita:\n", R_hat)


Epoch 0, costo: 46.4809
Epoch 10, costo: 46.3153
Epoch 20, costo: 46.1488
Epoch 30, costo: 45.9815
Epoch 40, costo: 45.8134
Epoch 50, costo: 45.6445
Epoch 60, costo: 45.4749
Epoch 70, costo: 45.3044
Epoch 80, costo: 45.1333
Epoch 90, costo: 44.9613
Epoch 100, costo: 44.7887
Epoch 110, costo: 44.6153
Epoch 120, costo: 44.4412
Epoch 130, costo: 44.2665
Epoch 140, costo: 44.0911
Epoch 150, costo: 43.9150
Epoch 160, costo: 43.7383
Epoch 170, costo: 43.5610
Epoch 180, costo: 43.3831
Epoch 190, costo: 43.2047
Epoch 200, costo: 43.0256
Epoch 210, costo: 42.8460
Epoch 220, costo: 42.6659
Epoch 230, costo: 42.4853
Epoch 240, costo: 42.3042
Epoch 250, costo: 42.1227
Epoch 260, costo: 41.9407
Epoch 270, costo: 41.7583
Epoch 280, costo: 41.5754
Epoch 290, costo: 41.3922
Epoch 300, costo: 41.2087
Epoch 310, costo: 41.0247
Epoch 320, costo: 40.8405
Epoch 330, costo: 40.6560
Epoch 340, costo: 40.4712
Epoch 350, costo: 40.2861
Epoch 360, costo: 40.1009
Epoch 370, costo: 39.9154
Epoch 380, costo: 39.72

: 